<a href="https://colab.research.google.com/github/nchadburn/Stat598-FinalProject/blob/main/MPLA%20Holdings%20Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# finanance data
import yfinance as yf

#visualization
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import seaborn as sns

#date manipulation
from datetime import datetime
import pandas as pd
import numpy as np

#Autoreg and Arima models
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

# statstical testing, plotting and decompositions
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error

In [2]:
start_date = '2020-06-01'
end_date = '2023-12-01'
etf_ticker = 'MLPA'
moving_average_list = [2, 5, 7, 14]
url = "https://raw.githubusercontent.com/Aditya-Patel/Stat598-FinalProject/main/mlpa_full-holdings.csv"


In [3]:
df_holdings = pd.read_csv(url)
df_holdings = df_holdings[(df_holdings['Name'] != 'OTHER PAYABLE & RECEIVABLES') & (df_holdings['Name'] != 'CASH')]

df_holdings[f'Market Value ($)'] = df_holdings[f'Market Value ($)'].str.replace(',', '').astype(float)
total_market_value = df_holdings[f'Market Value ($)'].sum()
df_holdings['Percentage Holdings By Value'] = (df_holdings[f'Market Value ($)'] / total_market_value)
df_holdings

,% of Net Assets,Ticker,Name,SEDOL,Market Price ($),Shares Held,Market Value ($),Percentage Holdings By Value
0,20.87,ET,ENERGY TRANSFER LP,B0WHXD2,13.33,"22,735,436.84",3.030634e+08,2.004323e-01
1,13.76,EPD,ENTERPRISE PRODU,2285388,26.25,"7,611,951.00",1.998137e+08,1.321477e-01
2,12.17,MPLX,MPLX LP,B847R56,35.87,"4,925,004.00",1.766599e+08,1.168348e-01
3,10.80,PAA,PLAINS ALL AMER,2311960,14.95,"10,486,658.00",1.567755e+08,1.036842e-01
4,10.32,WES,WESTERN MIDSTREA,BJ552C8,28.24,"5,303,028.00",1.497575e+08,9.904279e-02
5,9.47,ENLC,ENLINK MIDSTREAM LLC,BJTNDQ9,12.81,"10,731,120.00",1.374656e+08,9.091352e-02
6,8.26,CQP,CHENIERE ENERGY,B1VNH36,59.30,"2,023,409.00",1.199882e+08,7.935470e-02
7,6.53,NS,NUSTAR ENERGY LP,2738804,19.00,"4,991,913.00",9.484635e+07,6.272705e-02
8,5.91,HESM,HESS MIDSTREAM-A,BKPK695,30.70,"2,795,220.00",8.581325e+07,5.675298e-02
9,4.53,GEL,GENESIS ENERGY L.P.,2372257,12.32,"5,334,251.00",6.571797e+07,4.346287e-02


In [5]:


#start with etf data
all_stocks_data = yf.download(etf_ticker, start=start_date, end=end_date)
all_stocks_data['Ticker'] = etf_ticker
all_stocks_data.columns = [f'{etf_ticker}_{col}' if col != 'Ticker' else col for col in all_stocks_data.columns]

for ma in moving_average_list:
  all_stocks_data[f'{etf_ticker}_MA_{ma}'] = all_stocks_data[f'{etf_ticker}_Close'].rolling(window=ma).mean()
  all_stocks_data[f'{etf_ticker}_STD_{ma}'] = all_stocks_data[f'{etf_ticker}_Close'].rolling(window=ma).std()


# load all tickers part of that etf
for ticker in df_holdings['Ticker']:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    percentage_holding = df_holdings.loc[df_holdings['Ticker'] == ticker, 'Percentage Holdings By Value'].iloc[0]
    all_stocks_data[f'{ticker}_Percent_Holding'] = percentage_holding
    stock_data.columns = [f'{ticker}_{col}' if col != 'Ticker' else col for col in stock_data.columns]

    all_stocks_data = all_stocks_data.join(stock_data, how='outer')

all_stocks_data.fillna(0, inplace=True)

#adds every assets moving averages
for ticker in df_holdings['Ticker']:
    for ma in moving_average_list:
        all_stocks_data[f'{ticker}_MA_{ma}'] = all_stocks_data[f'{ticker}_Close'].rolling(window=ma).mean()
        all_stocks_data[f'{ticker}_STD_{ma}'] = all_stocks_data[f'{ticker}_Close'].rolling(window=ma).std()



# does some weighted and sum values of all the stock prices in the etf
weighted_values = pd.Series(0, index=all_stocks_data.index)
sum_values = pd.Series(0, index=all_stocks_data.index)


for ticker in df_holdings['Ticker']:
    weighted_values += all_stocks_data[f'{ticker}_Close'] * all_stocks_data[f'{ticker}_Percent_Holding']
    sum_values += all_stocks_data[f'{ticker}_Close']


all_stocks_data['Weighted Stock Value'] = weighted_values
all_stocks_data['Sum Stock Value'] = sum_values


all_stocks_data.reset_index(inplace=True)
all_stocks_data.tail()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

<ipython-input-5-f20895b37f34>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_stocks_data[f'{ticker}_STD_{ma}'] = all_stocks_data[f'{ticker}_Close'].rolling(window=ma).std()
<ipython-input-5-f20895b37f34>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_stocks_data[f'{ticker}_MA_{ma}'] = all_stocks_data[f'{ticker}_Close'].rolling(window=ma).mean()
<ipython-input-5-f20895b37f34>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

,Date,MLPA_Open,MLPA_High,MLPA_Low,MLPA_Close,MLPA_Adj Close,MLPA_Volume,Ticker,MLPA_MA_2,MLPA_STD_2,...,USDP_MA_2,USDP_STD_2,USDP_MA_5,USDP_STD_5,USDP_MA_7,USDP_STD_7,USDP_MA_14,USDP_STD_14,Weighted Stock Value,Sum Stock Value
878,2023-11-24,45.310001,45.450001,45.029999,45.200001,45.200001,32500,MLPA,45.200001,0.000000,...,0.2375,0.017678,0.1712,0.061707,0.190857,0.074434,0.278643,0.105843,25.016806,529.494997
879,2023-11-27,45.250000,45.330002,45.009998,45.250000,45.250000,48900,MLPA,45.225000,0.035355,...,0.2255,0.000707,0.1884,0.062812,0.178143,0.054673,0.266214,0.100582,24.955155,527.801000
880,2023-11-28,45.250000,45.259998,44.799999,45.250000,45.250000,91800,MLPA,45.250000,0.000000,...,0.2255,0.000707,0.2098,0.049686,0.186714,0.056926,0.254786,0.094980,25.002750,528.994998
881,2023-11-29,45.320000,45.660000,45.090000,45.500000,45.500000,63700,MLPA,45.375000,0.176777,...,0.2340,0.012728,0.2338,0.011862,0.201429,0.056145,0.245143,0.088113,25.287195,541.103003
882,2023-11-30,45.540001,46.119999,45.400002,46.119999,46.119999,73000,MLPA,45.809999,0.438405,...,0.2515,0.012021,0.2358,0.015547,0.221714,0.045650,0.237286,0.080717,25.661618,553.160002


In [9]:
all_stocks_data.shape
print(all_stocks_data.columns)

Index(['Date', 'MLPA_Open', 'MLPA_High', 'MLPA_Low', 'MLPA_Close',
       'MLPA_Adj Close', 'MLPA_Volume', 'Ticker', 'MLPA_MA_2', 'MLPA_STD_2',
       ...
       'USDP_MA_2', 'USDP_STD_2', 'USDP_MA_5', 'USDP_STD_5', 'USDP_MA_7',
       'USDP_STD_7', 'USDP_MA_14', 'USDP_STD_14', 'Weighted Stock Value',
       'Sum Stock Value'],
      dtype='object', length=318)


In [ ]:
split_percentage = .8


column_reference ="Close"
moving_average_list =[2,5,14]
adf_significance_level = .05
periods_to_test = [7, 14, 28,60, 90,180, 365]




all_stocks_data.reset_index(inplace=True)